In [1]:
import io
import sys
PATH = "D:/OneDrive - Universidad Tecnológica de Bolívar/Apps\MachineLearning-ISCO-A14A/"
DIR_DATA = PATH + "data/"
sys.path.append(PATH) if PATH not in list(sys.path) else None
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder 
from logic.text_processing import TextProcessing
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import classification_report, confusion_matrix, recall_score, log_loss
from sklearn.metrics import f1_score, accuracy_score, precision_score

In [2]:
tp = TextProcessing()
le = LabelEncoder()

In [3]:
data_train = pd.read_csv(DIR_DATA + 'tass2018_es_train.csv', sep=',')
data_train

,tweetid,user,content,date,lang,sentiment/polarity/value
0,768213876278165504,OnceBukowski,-Me caes muy bien \r\n-Tienes que jugar más pa...,2016-08-23 22:30:35,es,NONE
1,768213567418036224,anahorxn,@myendlesshazza a. que puto mal escribo\r\n\r\...,2016-08-23 22:29:21,es,N
2,768212591105703936,martitarey13,@estherct209 jajajaja la tuya y la d mucha gen...,2016-08-23 22:25:29,es,N
3,768221670255493120,endlessmilerr,Quiero mogollón a @AlbaBenito99 pero sobretodo...,2016-08-23 23:01:33,es,P
4,768221021300264964,JunoWTFL,Vale he visto la tia bebiendose su regla y me ...,2016-08-23 22:58:58,es,N
...,...,...,...,...,...,...
1003,814846333601320960,jdomperfects,Para mí mi mejor año fue el 2015,2016-12-30 14:51:19,es,P
1004,813731371076243461,GonzaloRoman_91,Hoy va a ser un dia muy largo...,2016-12-27 13:00:51,es,N
1005,818399956792905728,vaneftdallas,11:11 que no me encuentre mal,2017-01-09 10:12:09,es,NEU
1006,815715581878009858,carmen_ligero,Es que son muy bonitas jo,2017-01-02 00:25:24,es,P


In [4]:
data_test = pd.read_csv(DIR_DATA + 'tass2018_es_development.csv', sep=',')
data_test

,tweetid,user,content,date,lang,sentiment/polarity/value
0,770976639173951488,noseashetero,@noseashetero 1000/10 de verdad a ti que voy a...,2016-08-31 13:28:49,es,P
1,771092421866389508,Templelx,@piscolabisaereo @HistoriaNG @SPosteguillo las...,2016-08-31 21:08:54,es,P
2,771092111429083136,esskuu94,"Al final han sido 3h Bueno, mañana tengo fies...",2016-08-31 21:07:40,es,P
3,771092070572449796,__ariadna9,@Jorge_Ruiz14 yo no tengo tiempo para esas cos...,2016-08-31 21:07:30,es,N
4,771094192508600320,_cristtina15_,@_MissChaotic_ ves ese brillo? es un coso que ...,2016-08-31 21:15:56,es,N
...,...,...,...,...,...,...
501,757593585634213888,RubenRuCh,@Davorias Pero es que está muy rica,2016-07-25 15:09:20,es,P
502,757585725764362240,rayjaken,@narvalillo Eso es antiguo ya,2016-07-25 14:38:06,es,N
503,757097408304148480,Fuhyo__,Mi TL esta vacía a estas horas,2016-07-24 06:17:42,es,N
504,757971881077182464,xLytherinDovah,"Ya tengo uno, pero ese es más cheto @aweamasome.",2016-07-26 16:12:33,es,NEU


In [5]:
x_train = [tp.transformer(row) for row in data_train['content'].tolist()]
#y_train = le.fit_transform(data_train['sentiment/polarity/value'])
y_train = data_train['sentiment/polarity/value']
len(x_train), len(y_train)

(1008, 1008)

In [6]:
x_test = [tp.transformer(row) for row in data_test['content'].tolist()]
#y_test = le.fit_transform(data_test['sentiment/polarity/value'])
y_test = data_test['sentiment/polarity/value']
len(x_test), len(y_test)

(506, 506)

In [7]:
bow = CountVectorizer(analyzer='word', ngram_range=(2, 3))

In [8]:
x_train = bow.fit_transform(x_train)

In [9]:
x_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [10]:
x_test = bow.transform(x_test)

In [11]:
x_test.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [12]:
print('**Sample train:', sorted(Counter(y_train).items()))

**Sample train: [('N', 418), ('NEU', 133), ('NONE', 139), ('P', 318)]


In [13]:
print('**Sample test:', sorted(Counter(y_test).items()))

**Sample test: [('N', 219), ('NEU', 69), ('NONE', 62), ('P', 156)]


In [30]:
k_fold = ShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

In [31]:
ros_train = RandomOverSampler(random_state=1000)
x_train, y_train = ros_train.fit_resample(x_train, y_train)

In [32]:
print('**OverSample train:', sorted(Counter(y_train).items()))

**OverSample train: [('N', 418), ('NEU', 418), ('NONE', 418), ('P', 418)]


In [33]:
ros_test = RandomOverSampler(random_state=1000)
x_test, y_test = ros_test.fit_resample(x_test, y_test)

In [34]:
print('**OverSample test:', sorted(Counter(y_test).items()))

**OverSample test: [('N', 219), ('NEU', 219), ('NONE', 219), ('P', 219)]


In [35]:
softmax = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=10)

In [36]:
accuracies_scores = []
recalls_scores = []
precisions_scores = []
f1_scores = []

In [ ]:
for train_index, test_index in k_fold.split(x_train, y_train):
    data_train = x_train[train_index]
    target_train = y_train[train_index]
    
    data_test = x_train[test_index]
    target_test = y_train[test_index]

    softmax.fit(data_train, target_train)
    predict = softmax.predict(data_test)
    # Accuracy
    accuracy = accuracy_score(target_test, predict)
    accuracies_scores.append(accuracy)
    # Recall
    recall = recall_score(target_test, predict, average='macro')
    recalls_scores.append(recall)
    # Precision
    precision = precision_score(target_test, predict, average='weighted')
    precisions_scores.append(precision)
    # F1
    f1 = f1_score(target_test, predict, average='weighted')
    f1_scores.append(f1)

In [ ]:
average_recall = round(np.mean(recalls_scores) * 100, 2)
average_precision = round(np.mean(precisions_scores) * 100, 2)
average_f1 = round(np.mean(f1_scores) * 100, 2)
average_accuracy = round(np.mean(accuracies_scores) * 100, 2)

In [ ]:
y_predict = []
for features in x_test:
    features = features.reshape(1, -1)
    value = softmax.predict(features)[0]
    y_predict.append(value)

classification = classification_report(y_test, y_predict)
confusion = confusion_matrix(y_predict, y_test)

In [ ]:
output_result = {'F1-score': average_f1, 'Accuracy': average_accuracy, 'Recall': average_recall, 
                 'Precision': average_precision, 'Classification Report\n': classification, 
                 'Confusion Matrix\n': confusion}

In [ ]:
for item, val in output_result.items():
    print('{0} {1}'.format(item, val))